# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv = "city_df.csv"
city_df = pd.read_csv("city_df.csv")
city_df.head()

,Unnamed: 0,CityID,City,Humidity,Temp,Latitude,Cloudiness,WindSpeed,MaxTemp,Longitude,LatLong,CountryCode
0,0,2027296,Aykhal,65,-40.00,66.00,0,3.58,-40.0,111.50,"{'lon': 111.5, 'lat': 66}",RU
1,1,3874787,Punta Arenas,81,50.00,-53.15,75,13.87,50.0,-70.92,"{'lon': -70.92, 'lat': -53.15}",CL
2,2,2270385,Camacha,82,57.97,33.08,20,10.29,59.0,-16.33,"{'lon': -16.33, 'lat': 33.08}",PT
3,3,2155415,New Norfolk,67,60.24,-42.78,75,9.17,62.6,147.06,"{'lon': 147.06, 'lat': -42.78}",AU
4,4,4030556,Rikitea,76,79.30,-23.12,2,11.79,79.3,-134.97,"{'lon': -134.97, 'lat': -23.12}",PF


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_df[["Latitude","Longitude"]]
humidity = city_df["Humidity"].astype(float)


In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=10, point_radius=1)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
maxtemp_df = city_df[(city_df['MaxTemp'] < 80) & (city_df['MaxTemp'] > 70)]
windspeed_df = maxtemp_df[(maxtemp_df['WindSpeed'] < 20)]
perfect_df = windspeed_df[(windspeed_df['Cloudiness'] ==0)].dropna()
perfect_df

,Unnamed: 0,CityID,City,Humidity,Temp,Latitude,Cloudiness,WindSpeed,MaxTemp,Longitude,LatLong,CountryCode
51,51,3897774,Buin,56,73.38,-33.73,0,13.87,75.20,-70.75,"{'lon': -70.75, 'lat': -33.73}",CL
80,80,2074865,Carnarvon,47,78.80,-24.87,0,17.22,78.80,113.63,"{'lon': 113.63, 'lat': -24.87}",AU
96,96,2138555,Poum,75,79.72,-20.23,0,12.97,79.72,164.02,"{'lon': 164.02, 'lat': -20.23}",NC
157,157,3443061,Chui,82,70.14,-33.70,0,9.86,70.14,-53.46,"{'lon': -53.46, 'lat': -33.7}",UY
160,160,2137773,Vao,74,77.20,-22.67,0,15.43,77.20,167.48,"{'lon': 167.48, 'lat': -22.67}",NC
241,241,1717787,Catuday,70,75.60,16.29,0,6.89,75.60,119.81,"{'lon': 119.81, 'lat': 16.29}",PH
307,307,3584003,Puerto El Triunfo,65,78.80,13.28,0,3.36,78.80,-88.55,"{'lon': -88.55, 'lat': 13.28}",SV
385,385,3838874,Rio Cuarto,88,71.60,-33.13,0,10.29,71.60,-64.35,"{'lon': -64.35, 'lat': -33.13}",AR
387,387,2301217,Ejura,67,76.69,7.39,0,2.68,76.69,-1.36,"{'lon': -1.36, 'lat': 7.39}",GH
474,474,3443341,Carmelo,83,77.00,-33.99,0,12.75,77.00,-58.29,"{'lon': -58.29, 'lat': -33.99}",UY


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame(columns= ['hotel_name','CityID'])

for index,row in perfect_df.iterrows():
    target_coordinates = f"{row['Latitude']},{row['Longitude']}"
    target_search = "Hotels"
    target_radius = 5000
    target_type = "lodging"
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    response = response.json()

    if response['status'] =='OK':
        hotel_df = hotel_df.append({'hotel_name' : response["results"][0]["name"],'CityID' : row['CityID']}, ignore_index=True)
    else:
        print('No Hotel Located')
hotel_df

No Hotel Located
No Hotel Located


,hotel_name,CityID
0,Motel INSIDE,3897774
1,Hospitality Carnarvon,2074865
2,Hotel Internacional,3443061
3,Hotel Kou-Bugny,2137773
4,Hotel G Beach Resort and Restobar,1717787
5,Hotel Garden House,3838874
6,Collinmay Hotel,2301217
7,Carmelo Resort & Spa,3443341
8,Palm Spring Hotel,3379507


In [10]:
BestHotels = pd.merge(hotel_df, perfect_df, on="CityID")
BestHotels.set_index("CityID", inplace=True)
BestHotels

,hotel_name,Unnamed: 0,City,Humidity,Temp,Latitude,Cloudiness,WindSpeed,MaxTemp,Longitude,LatLong,CountryCode
CityID,,,,,,,,,,,,
3897774,Motel INSIDE,51,Buin,56,73.38,-33.73,0,13.87,75.20,-70.75,"{'lon': -70.75, 'lat': -33.73}",CL
2074865,Hospitality Carnarvon,80,Carnarvon,47,78.80,-24.87,0,17.22,78.80,113.63,"{'lon': 113.63, 'lat': -24.87}",AU
3443061,Hotel Internacional,157,Chui,82,70.14,-33.70,0,9.86,70.14,-53.46,"{'lon': -53.46, 'lat': -33.7}",UY
2137773,Hotel Kou-Bugny,160,Vao,74,77.20,-22.67,0,15.43,77.20,167.48,"{'lon': 167.48, 'lat': -22.67}",NC
1717787,Hotel G Beach Resort and Restobar,241,Catuday,70,75.60,16.29,0,6.89,75.60,119.81,"{'lon': 119.81, 'lat': 16.29}",PH
3838874,Hotel Garden House,385,Rio Cuarto,88,71.60,-33.13,0,10.29,71.60,-64.35,"{'lon': -64.35, 'lat': -33.13}",AR
2301217,Collinmay Hotel,387,Ejura,67,76.69,7.39,0,2.68,76.69,-1.36,"{'lon': -1.36, 'lat': 7.39}",GH
3443341,Carmelo Resort & Spa,474,Carmelo,83,77.00,-33.99,0,12.75,77.00,-58.29,"{'lon': -58.29, 'lat': -33.99}",UY
3379507,Palm Spring Hotel,560,Bartica,94,75.20,6.40,0,4.70,75.20,-58.62,"{'lon': -58.62, 'lat': 6.4}",GY


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{CountryCode}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in BestHotels.iterrows()]
locations = BestHotels[["Latitude", "Longitude"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display Map

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig
plt.savefig('Hotel Map')

<Figure size 432x288 with 0 Axes>